<a href="https://colab.research.google.com/github/xloem/apifudge/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install adapter-transformers accelerate bitsandbytes
!git clone https://github.com/xloem/apifudge; cd apifudge; git pull; git checkout main #15f5b4bc5cbc2cf28c52dff3a997d9d5e539c71a
!nvidia-smi
!ln -vsf apifudge/* .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'apifudge' already exists and is not an empty directory.
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 1), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/xloem/apifudge
   2387311..75f8b99  main       -> origin/main
Updating 2387311..75f8b99
Fast-forward
 apifudge1/__init__.py | 24 ++++++++++++------------
 1 file changed, 12 insertions(+), 12 deletions(-)
Already on 'main'
Your branch is up to date with 'origin/main'.
Sat Jan  7 23:23:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persisten

In [2]:
# use drive to cache downloads
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/colab/.cache
!rm -rf $HOME/.cache
!ln -s /content/drive/MyDrive/colab/.cache $HOME/.cache

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import transformers
print('loading model ...')
pipeline = transformers.pipeline(
    'text-generation', 'bigscience/bloomz-3b', 
    model_kwargs=dict(device_map='auto'))
print('loaded model')

loading model ...
loaded model


In [4]:
import apifudge1 as af1
npapi = af1.API('numpy')
npapi.set_example(
'''Construct an array B that repeats array Y 4 times along the last dimension.''',
'''B = np.tile(Y, 4)'''
)
npapi.set_example(
'''Swap the last two axes of an array D.''',
'''D = D.swapaxes(-2, -1)'''
)
npapi.set_example(
'''Sort an array along the third axis.''',
'''array.sort(3)'''
)
for name, text in npapi.get_examples().items():
    print(f'{name}: {repr(text)}')
generator = af1.Generator(pipeline)
print('Augmenting ...')
for prompt, result in generator.augment(npapi):
  print(f'{prompt}: {repr(result)}')
print('Inferring:')
prompt = 'Reverse an array C.'
output = generator(npapi, prompt)
print(f'{prompt}: {repr(output)}')

Sort an array along the third axis.: 'array.sort(3)'
Swap the last two axes of an array D.: 'D = D.swapaxes(-2, -1)'
Construct an array B that repeats array Y 4 times along the last dimension.: 'B = np.tile(Y, 4)'
Augmenting ...
Convert a string to an axised tensor Y.: 'Y = np.array(Y).T'
Create an array B from a vector of zeros: 'B = np.zeros((4,))'
Averaging elements of B along the last axis (optional).: 'B = np.mean(B, axis=0)'
Apply the last two axes of array Y to B. B = B.swapaxes(-2, -1): 'B'
Construct 2D array of integers x, y.: 'X = np.array([[1, 2, 3], [4, 5, 6]])'
A 2D array can share two dimensional elements.: 'A = np.concatenate([np.array(X), np.array(Y)])'
Construct a multidimensional array of shape (X + Y + Z) D = [[X for y in range(X)] for z in range(X) for x in range(X)]: 'D'
Perform a mean shift. D.: 'D.mean()'
Inferring:
Reverse an array C.: 'C = np.flipud(C)'
